<a href="https://colab.research.google.com/github/trashchenkov/gigachat_tutorials/blob/main/%D0%97%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузчики документов для RAG и других цепочек GigaChain/LangChain

Создание вопросно-ответных систем или иных приложений, где большая языковая модель взаимодействует с контекстом, нужны подготовленные тексты из внешних источников. Очень часто источниками этих текстов являются файлы различных форматов. В этом туториале мы познакомимся с работой некоторых загрузчиков, которые есть в библиотеке GigaChain/LangChain, позволяющих извлекать тексты из файлов. Мы рассмотрим примеры как обычных текстовых файлов, так и табличных документов. Загрузчики представляют собой интеграции внешних библиотек. В результате при работе с любыми источниками используется один и тот же интерфейс и единый формат выдаваемых на выходе данных, что облегчает разработку приложений.

На основе извлеченных текстовых данных мы создадим векторные базы данных с эмбеддингами, а затем сделаем ряд запросов к этим базам данных, чтобы проверить, насколько возвращаемые фрагменты релевантны запросам.

Давайте приступим!

## Установим необходимые библиотеки

In [1]:
!pip install gigachain gigachain-community unstructured[all-docs] faiss-cpu sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 995.3/995.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 56.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 43.

## Текстовые данные
В этом разделе мы рассмотрим загрузчики для текстовых данных: PDF и DOCX файлов.

В арсенале GigaChain есть несколько загрузчиков под каждый из этих типов файлов. Мы воспользуемся `UnstructuredWordDocumentLoader` и `UnstructuredPDFLoader` для DOCX и PDF соответственно. Другие варианты загрузчиков можно посмотреть по ссылкам:
- [DOCX](https://python.langchain.com/v0.1/docs/integrations/document_loaders/microsoft_word/);
- [PDF](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/).

В качестве текстов используем статью:

*Голиков А.А., Акимов Д.А., Романовский М.С., Тращенков С.В. — Аспекты создания корпоративной вопросно-ответной системы с использованием генеративных предобученных языковых моделей // Litera. – 2023. – No 12. – С. 190 - 205. DOI: 10.25136/2409-8698.2023.12.69353 EDN: FSTHRW URL: https://nbpublish.com/library_read_article.php?id=69353*


### Создание фрагментов на основе PDF

Для подготовки фрагментов используем `RecursiveCharacterTextSplitter`. Зададим приблизительный размер фрагмента и количество символов перекрытия. Сплиттер можно передать в метод `load_and_split()`. На выходе получим список документов `Document`.

In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                              chunk_overlap=100)
pdf_file = 'aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.pdf'
loader = UnstructuredPDFLoader(pdf_file)
splitted_data = loader.load_and_split(text_splitter)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Посмотрим сколько фрагментов получилось.

In [4]:
len(splitted_data)

47

Выведем на экран два соседних фрагмента, чтобы показать перекрытие.

In [5]:
splitted_data[10]

Document(page_content='. Автор вышеупомянутой диссертации даже создалотдельный сервис (https://demo.caire.ust.hk/), работающий как генеративная вопросно-ответная система по большому количеству статей о коронавирусе. Однако, отдаваядолжное автору вышеупомянутых диссертации и сервиса, стоит сказать, что в настоящеевремя, в частности, за счет появившихся уже после релиза ChatGPT фреймворковLlamaIndex, а также возможности использования фреймворка Haystack совместно с GPT-3 для построения генеративной вопросно-ответной системы, создание подобной системыстало значительно проще и доступнее, а потому сравнение фреймворков и их настроекактуально и представляет значительный интерес.2 Расширение возможностей применения больших языковых моделей путемиспользования индексации документовОсновным способом построения вопросно-ответных систем является использованиеретривера для определения наиболее релевантных запросу частей текста, а затемсинтез ответа из найденных частей текста с использованием т.н. р

In [6]:
splitted_data[11]

Document(page_content='запросу частей текста, а затемсинтез ответа из найденных частей текста с использованием т.н. ридера (дляэкстрактивной вопросно-ответной системы) или генератора (для генеративной вопросно-ответной системы).При этом для более эффективного поиска целесообразно предварительно провести т.н.индексацию документа или набора документов, по которым предполагается производитьпоиск. Под индексацией подразумевается выявление и сохранение некоей ключевойинформации о частях документов, с помощью которой в дальнейшем удобно определить,насколько та или иная часть текста соответствует поисковому запросу (рисунок 1).', metadata={'source': 'aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.pdf'})

### Создание векторных представлений и векторного хранилища

В этом туториале опять используем модель [`paraphrase-multilingual-mpnet-base-v2`](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2) для векторизации фрагментов.

В качестве хранилища опять используем `FAISS`. Помимо этой векторной БД в GigaChain/LangChain есть большое количество интеграций с векторными хранилищами, как локальными так и облачными.
Перечень доступных векторных хранилищ можно посмотреть [здесь](https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS

In [8]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(splitted_data, embedding=embedding)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 31.7 s, sys: 9.34 s, total: 41 s
Wall time: 55.3 s


### Поиск по векторной БД релевантных фрагментов для запросов пользователя
Зададим несколько вопросов по тексту статьи.

- Какой размер чанков показал наибольшую точность ответов при использовании фреймворка Haystack?
- Какие метрики использовались для оценки эффективности использования генеративных предобученных языковых моделей в статье?
- Имя какого английского писателя упомянается в статье?

In [9]:
q1 = 'Какой размер чанков показал наибольшую точность ответов при использовании фреймворка Haystack?'
q2 = 'Какие метрики использовались для оценки эффективности использования генеративных предобученных языковых моделей в статье?'
q3 = 'Имя какого английского писателя упоминается в статье?'

#### Вопрос 1
Какой размер чанков показал наибольшую точность ответов при использовании фреймворка Haystack?

#### Правильный ответ
Размер чанков 100, 200 и 1000 токенов показал одинаковую наибольшую точность ответов (0.7) при использовании фреймворка Haystack.

In [10]:
vector_store.similarity_search(q1)

[Document(page_content='при помощи фреймворка Haystack прилучших настройках позволяет получить несколько бо́льшую точность ответов, чемиспользование фреймворка LlamaIndex (0.7 против 0.67 при лучших настройках). 3.Использование фреймворка Haystack более инвариантно относительно точности ответовс точки зрения количества токенов в чанке. 4. В среднем использование фреймворкаHaystack более затратно по числу токенов (примерно в 4 раза), чем фреймворкаLlamaIndex. 5. Режимы генерации ответа «create and refine» и «tree summarize» дляфреймворка LlamaIndex являются примерно одинаковыми с точки зрения точностиполучаемых ответов, однако для режима «tree summarize» требуется больше токенов.Ключевые слова:генеративные языковые модели, информационная поисковая система, вопросно-ответная система, индексация, Haystack, LlamaIndex, чанк, точность, токен, ретривер10.25136/2409-8698.2023.12.69353Litera, 2023 - 12191', metadata={'source': 'aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolz

#### Вопрос 2
Какие метрики использовались для оценки эффективности использования генеративных предобученных языковых моделей в статье?

#### Правильный ответ
Для оценки эффективности использования генеративных предобученных языковых моделей использовалась метрика EM (exact match).

In [11]:
vector_store.similarity_search(q2)

[Document(page_content='фреймворкомLlamaIndex, однако требует использования в среднем несколько бо́льшего числатокенов. В статье использовался сравнительный анализ для оценки эффективностииспользования генеративных предобученных языковых моделей в корпоративныхвопросно-ответных системах с помощью фреймворков Haystack и Llamaindex. Оценкаполученных результатов осуществлялась с использованием метрики EM (exact match).Основными выводами проведенного исследования по созданию вопросно-ответныхсистем с использованием генеративных предобученных языковых моделей являются: 1.Использование иерархической индексации на текущий момент чрезвычайно затратно сточки зрения числа используемых токенов (около 160000 токенов для иерархическойиндексации против 30000 токенов в среднем для последовательной индексации),поскольку ответ генерируется путем последовательной обработки родительских идочерних узлов. 2. Обработка информации при помощи фреймворка Haystack прилучших настройках позволяет получить несколь

#### Вопрос 3
Имя какого английского писателя упоминается в статье?

#### Правильный ответ
В статье упоминается английский писатель Джордж Гордон Байрон (лорд Байрон).

In [12]:
vector_store.similarity_search(q3)

[Document(page_content='.Тема рецензируемой статьи, безусловно, является актуальной; автор данного труда10.25136/2409-8698.2023.12.69353Litera, 2023 - 12203', metadata={'source': 'aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.pdf'}),
 Document(page_content='слов: для каждой части текста сохраняются ключевые термины, о которых идет речь, идалее при поисковом запросе будет выполнено сопоставление терминов запроса исохраненных ключевых слов участков текста. Так, при поиске по странице «Википедии»,посвященной лорду Байрону, при поисковом запросе «В каком году родился лордБайрон?» по ключевым словам «рождение», «родиться» ретривером может быть найденучасток текста «Джордж Гордон Байрон родился 22 января 1788 года». И далее задачаридера будет заключаться в извлечении из найденного участка текста требуемойинформации – т.е. «1788 год» в данном случае.Очевидно, что подобный пример индексации с использованием ключевых слов

### Создание фрагментов на основе DOCX
Используем эту же статью, но в формате DOCX. В качестве загрузчика у нас будет `UnstructuredWordDocumentLoader`.

In [13]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
loader = UnstructuredWordDocumentLoader('aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.docx')
splitted_docx = loader.load_and_split(text_splitter)

Проверим, сколько вышло фрагментов.

In [14]:
len(splitted_docx)

45

Видим, что количество фрагментов не совпадает. Сравним содержимое фрагментов из pdf и docx.

In [15]:
splitted_data[10]

Document(page_content='. Автор вышеупомянутой диссертации даже создалотдельный сервис (https://demo.caire.ust.hk/), работающий как генеративная вопросно-ответная система по большому количеству статей о коронавирусе. Однако, отдаваядолжное автору вышеупомянутых диссертации и сервиса, стоит сказать, что в настоящеевремя, в частности, за счет появившихся уже после релиза ChatGPT фреймворковLlamaIndex, а также возможности использования фреймворка Haystack совместно с GPT-3 для построения генеративной вопросно-ответной системы, создание подобной системыстало значительно проще и доступнее, а потому сравнение фреймворков и их настроекактуально и представляет значительный интерес.2 Расширение возможностей применения больших языковых моделей путемиспользования индексации документовОсновным способом построения вопросно-ответных систем является использованиеретривера для определения наиболее релевантных запросу частей текста, а затемсинтез ответа из найденных частей текста с использованием т.н. р

In [16]:
splitted_docx[10]

Document(page_content='или иным способом до объема менее 4000 токенов, однако, очевидно, что значительная часть информации в таком случае будет потеряна. Во многих случаях наиболее привлекательным способом решения вопроса является индексация', metadata={'source': 'aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.docx'})

In [17]:
splitted_docx[12]

Document(page_content='Стоит отметить, что исследованиям в области вопросно-ответных систем посвящено множество публикаций, однако в большинстве своем они посвящены экстрактивным вопросно-ответным системам, поскольку достаточно качественные генеративные большие языковые модели появились позже. Из наиболее свежих и релевантных научных материалов, касающихся в том числе генеративных вопросно-ответных систем, можно выделить статьи [4-6] и диссертацию [7]. Автор вышеупомянутой диссертации даже создал отдельный сервис (https://demo.caire.ust.hk/), работающий как генеративная вопросно- ответная система по большому количеству статей о коронавирусе. Однако, отдавая должное автору вышеупомянутых диссертации и сервиса, стоит сказать, что в настоящее время, в частности, за счет появившихся уже после релиза ChatGPT фреймворков LlamaIndex, а также возможности использования фреймворка Haystack совместно с GPT- 3 для построения генеративной вопросно-ответной системы, создание подобной системы стало з

In [22]:
docs = embedding.embed_documents([splitted_data[10].page_content, splitted_docx[12].page_content])

In [23]:
docs

[[0.008951683528721333,
  0.13602659106254578,
  -0.01078121643513441,
  0.005982115399092436,
  -0.023294484242796898,
  0.06577566266059875,
  0.15637962520122528,
  0.05735860392451286,
  0.03467786684632301,
  -0.029316941276192665,
  0.16848066449165344,
  0.055822547525167465,
  0.003838876262307167,
  -0.05944221466779709,
  0.08486774563789368,
  -0.10907694697380066,
  0.06572281569242477,
  0.0546577088534832,
  -0.025797586888074875,
  0.05429495871067047,
  0.11603781580924988,
  -0.019520800560712814,
  0.05233471095561981,
  0.054645687341690063,
  -0.06036115810275078,
  -0.060792334377765656,
  -0.052016496658325195,
  -0.03358055278658867,
  0.07728233933448792,
  -0.03454596549272537,
  0.030566954985260963,
  0.1216118112206459,
  0.039400260895490646,
  0.06916099786758423,
  0.07300259172916412,
  -0.005192926619201899,
  -0.024102987721562386,
  0.06088732182979584,
  -0.046460650861263275,
  0.04642333462834358,
  0.3130835294723511,
  0.12919700145721436,
  0.03

In [24]:
import numpy as np
cosine_similarity = np.dot(docs[0], docs[1]) / (np.linalg.norm(docs[0]) * np.linalg.norm(docs[1]))

print(f"Косинусное сходство: {cosine_similarity}")

Косинусное сходство: 0.730230684886262


## Создание векторных представлений и векторного хранилища

Создадим векторное хранилище, но уже из фрагментов docx-файла. Затем зададим такие же вопросы.

In [25]:
vector_store_docx = FAISS.from_documents(splitted_docx, embedding=embedding)

#### Вопрос 1
Какой размер чанков показал наибольшую точность ответов при использовании фреймворка Haystack?

#### Правильный ответ
Размер чанков 100, 200 и 1000 токенов показал одинаковую наибольшую точность ответов (0.7) при использовании фреймворка Haystack.

In [26]:
vector_store_docx.similarity_search(q1)

[Document(page_content='получить несколько бо́льшую точность ответов, чем использование фреймворка LlamaIndex (0.7 против 0.67 при лучших настройках). 3. Использование фреймворка Haystack более инвариантно относительно точности ответов с точки зрения количества токенов в чанке. 4. В среднем использование фреймворка Haystack более затратно по числу токенов (примерно в 4 раза), чем фреймворка LlamaIndex. 5. Режимы генерации ответа «create and refine» и «tree summarize» для фреймворка LlamaIndex являются примерно одинаковыми с точки зрения точности получаемых ответов, однако для режима «tree summarize» требуется больше токенов.', metadata={'source': 'aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.docx'}),
 Document(page_content='Рисунок 7: Количество затраченных токенов ответов при различных настройках вопросно-ответной системы (синий цвет – фреймворк Haystack, оранжевый – LlamaIndex, отсутствие штриховки – режим ген

#### Вопрос 2
Какие метрики использовались для оценки эффективности использования генеративных предобученных языковых моделей в статье?

#### Правильный ответ
Для оценки эффективности использования генеративных предобученных языковых моделей использовалась метрика EM (exact match).

In [27]:
vector_store_docx.similarity_search(q2)

[Document(page_content='бо́льшего числа токенов. В статье использовался сравнительный анализ для оценки эффективности использования генеративных предобученных языковых моделей в корпоративных вопросно-ответных системах с помощью фреймворков Haystack и Llamaindex. Оценка полученных результатов осуществлялась с использованием метрики EM (exact match). Основными выводами проведенного исследования по созданию вопросно-ответных систем с использованием генеративных предобученных языковых моделей являются: 1. Использование иерархической индексации на текущий момент чрезвычайно затратно с точки зрения числа используемых токенов (около 160000 токенов для иерархической индексации против 30000 токенов в среднем для последовательной индексации), поскольку ответ генерируется путем последовательной обработки родительских и дочерних узлов. 2. Обработка информации при помощи фреймворка Haystack при лучших настройках позволяет получить несколько бо́льшую точность ответов, чем использование фреймворка L

#### Вопрос 3
Имя какого английского писателя упоминается в статье?

#### Правильный ответ
В статье упоминается английский писатель Джордж Гордон Байрон (лорд Байрон).

In [28]:
vector_store_docx.similarity_search(q3)

[Document(page_content='В связи с политикой двойного слепого рецензирования личность рецензента не раскрывается.\n\nСо списком рецензентов издательства можно ознакомиться здесь.\n\nТема рецензируемой статьи, безусловно, является актуальной; автор данного труда', metadata={'source': 'aspekty-sozdaniya-korporativnoy-voprosno-otvetnoy-sistemy-s-ispolzovaniem-generativnyh-predobuchennyh-yazykovyh-modeley.docx'}),
 Document(page_content='Litera\n\nПравильная ссылка на статью:\n\nГоликов А.А., Акимов Д.А., Романовский М.С., Тращенков С.В. — Аспекты создания корпоративной вопросно-ответной системы с использованием генеративных предобученных языковых моделей // Litera. – 2023. – № 12. – С. 190 - 205. DOI: 10.25136/2409-8698.2023.12.69353 EDN: FSTHRW URL:\n\nhttps://nbpublish.com/library_read_article.php?id=69353\n\nАспекты создания корпоративной вопросно-ответной системы с использованием генеративных предобученных языковых моделей\n\nГоликов Алексей Александрович\n\nаспирант, Отделение филолог

## Табличные данные
Теперь рассмотрим стандартные форматы табличных файлов: CSV и XLSX.

Для этого используем загрузчики `CSVLoader` и `UnstructuredExcelLoader`.

В качестве табличного файла используем документ ["Динамика уровня и структуры затрат организаций на рабочую силу"](https://rosstat.gov.ru/labour_costs#) с сайта Росстата.

Файл представляет собой несколько таблиц в основном с количественными данными. Поскольку файл изначально в формате XLSX, сделаем из него и документы в формате CSV. Так как в XLSX-файле несколько листов, придется создать отдельный CSV-файл под каждый лист.

In [29]:
import pandas as pd

# Загрузите Excel файл
file_path = '/content/din-zatrat-rs.xlsx'

# Читаем все листы из Excel файла
sheets = pd.read_excel(file_path, sheet_name=None)

# Сохраняем каждый лист как отдельный CSV файл
for sheet_name, df in sheets.items():
    csv_file = f"{sheet_name}.csv"
    df.to_csv(csv_file, index=False)
    print(f"Сохранено: {csv_file}")


Сохранено: Содержание.csv
Сохранено: 1..csv
Сохранено: 2..csv
Сохранено: 3..csv
Сохранено: 4..csv
Сохранено: 5..csv
Сохранено: 6..csv
Сохранено: Основные понятия и определения.csv


### Создание списка csv-файлов и загрузка
Поскольку загрузчик `CSVLoader` принимает документы по одному, делаем загрузку в цикле.

In [30]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from glob import glob
files = glob('*.csv')

splitted_csv = []
for f in files:
  loader = CSVLoader(f)
  splitted_csv.extend(loader.load())

Проверим, сколько фрагментов получилось.

In [31]:
len(splitted_csv)

140

Посмотрим отдельный фрагмент.

In [32]:
splitted_csv[139]

Document(page_content='К содержанию: Деятельность административная и сопутствующие дополнительные услуги\nUnnamed: 1: 36846.5\nUnnamed: 2: 62926.9\nUnnamed: 3: 223.5\nUnnamed: 4: 392.9\nUnnamed: 5: 247.2\nUnnamed: 6: 434.9\nUnnamed: 7: 90.4\nUnnamed: 8: 90.3\nUnnamed: 9: 79.4\nUnnamed: 10: 80.5\nUnnamed: 11: 20.6\nUnnamed: 12: 19.5', metadata={'source': '4..csv', 'row': 19})

### Создание векторных представлений и векторного хранилища
Поместим фрагменты в базу данных.

In [33]:
vector_store_csv = FAISS.from_documents(splitted_csv, embedding=embedding)

### Поиск по векторной БД релевантных фрагментов для запросов пользователя
Зададим несколько вопросов по данным таблиц.
- Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?
- Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?
- В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?

In [34]:
q4 = 'Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?'
q5 = 'Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?'
q6 = 'В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?'

#### Вопрос 4
Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?

#### Правильный ответ
В 2021 году расходы на социальную защиту работников в добыче полезных ископаемых составляли 22,6%.

In [35]:
vector_store_csv.similarity_search(q4)

[Document(page_content='К содержанию: Затраты организаций на рабочую силу - это сумма вознаграждений работников (среднесписочной численности) в денежной и неденежной формах за отработанное и неотработанное время, расходы организации, связанные с обеспечением работников жильем, профессиональным обучением, культурно-бытовым обслуживанием, социальной защитой, включая расходы на пенсионное, медицинское и другие виды страхования, командировочные расходы, а также налоги и сборы, связанные с использованием наемной рабочей силы. В составе затрат на рабочую силу можно выделить две группы затрат: прямые и косвенные.\nПрямые затраты включают расходы, направленные организациями непосредственно на удовлетворение потребностей работников: заработная плата, расходы на жилье, выплаты социального характера, расходы на профессиональное обучение и культурно-бытовое обслуживание, оплата проезда и другие расходы, не отнесенные к приведенным классификационным группам.\nК косвенным затратам относятся средства

#### Вопрос 5
Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?

#### Правильный ответ
В 2005 году заработная плата составляла 76,6% от общих затрат на рабочую силу по всем обследованным видам экономической деятельности.

In [36]:
vector_store_csv.similarity_search(q5)

[Document(page_content='К содержанию: Затраты организаций на рабочую силу - это сумма вознаграждений работников (среднесписочной численности) в денежной и неденежной формах за отработанное и неотработанное время, расходы организации, связанные с обеспечением работников жильем, профессиональным обучением, культурно-бытовым обслуживанием, социальной защитой, включая расходы на пенсионное, медицинское и другие виды страхования, командировочные расходы, а также налоги и сборы, связанные с использованием наемной рабочей силы. В составе затрат на рабочую силу можно выделить две группы затрат: прямые и косвенные.\nПрямые затраты включают расходы, направленные организациями непосредственно на удовлетворение потребностей работников: заработная плата, расходы на жилье, выплаты социального характера, расходы на профессиональное обучение и культурно-бытовое обслуживание, оплата проезда и другие расходы, не отнесенные к приведенным классификационным группам.\nК косвенным затратам относятся средства

#### Вопрос 6
В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?

#### Правильный ответ
Наибольший процент заработной платы в обрабатывающей промышленности был зафиксирован в 2007 году и составил 79,5%.

In [37]:
vector_store_csv.similarity_search(q6)

[Document(page_content='К содержанию: Затраты организаций на рабочую силу - это сумма вознаграждений работников (среднесписочной численности) в денежной и неденежной формах за отработанное и неотработанное время, расходы организации, связанные с обеспечением работников жильем, профессиональным обучением, культурно-бытовым обслуживанием, социальной защитой, включая расходы на пенсионное, медицинское и другие виды страхования, командировочные расходы, а также налоги и сборы, связанные с использованием наемной рабочей силы. В составе затрат на рабочую силу можно выделить две группы затрат: прямые и косвенные.\nПрямые затраты включают расходы, направленные организациями непосредственно на удовлетворение потребностей работников: заработная плата, расходы на жилье, выплаты социального характера, расходы на профессиональное обучение и культурно-бытовое обслуживание, оплата проезда и другие расходы, не отнесенные к приведенным классификационным группам.\nК косвенным затратам относятся средства

### Загрузка табличных данных из xlsx-файла
Для сравнения мы выполним разделение на фрагменты тремя способами:
- используем сплиттер из текстовых примеров (1000 символов на фрагмент и 100 символов на перекрытие);
- применим сплиттер по умолчанию;
- зададим параметр `mode='elements'` в загрузчике.

При первом варианте настроек загрузчика получаем мелкие фрагменты.

In [38]:
from langchain_community.document_loaders import UnstructuredExcelLoader

loader = UnstructuredExcelLoader('din-zatrat-rs.xlsx')
splitted_xlsx = loader.load_and_split(text_splitter)

Посмотрим, сколько всего получилось.

In [39]:
len(splitted_xlsx)

24

И отдельно посмотрим один из них.

In [40]:
splitted_xlsx[5]

Document(page_content='Затраты на рабочую силу1)\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n100\n100.0\n100.0\n100.0\n\n\nв том числе:', metadata={'source': 'din-zatrat-rs.xlsx'})

Теперь используем настройки по умолчанию.

In [41]:
splitted_xlsx1 = loader.load_and_split()

Получим другой список с более крупными фрагментами.

In [42]:
len(splitted_xlsx1)

6

Посмотрим отдельный фрагмент.

In [43]:
splitted_xlsx1[3]

Document(page_content='Среднемесячные затраты, рублей\n\nСредние затраты на один оплаченный час, рублей\n\nСредние затраты на один отработанный час, рублей\n\nОтношение отработанного времени к оплаченному,  процентов\n\nДоля прямых затрат на рабочую силу в общих затратах на рабочую силу, процентов\n\nДоля косвенных затрат на рабочую силу в общих затратах на рабочую силу, процентов\n\n\n\n\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n\n\nВсего по обследуемым видам экономической деятельности\n60234.7\n90058.9\n374.3\n563.3\n419.4\n630.4\n89.2\n89.4\n80\n80.2\n20\n19.8\n\n\nДобыча полезных ископаемых\n104076.1\n140967.4\n622.5\n845.1\n722.3\n973.6\n86.2\n86.8\n80.3\n80.0\n19.7\n20.0\n\n\nОбрабатывающие производства\n54154.2\n76419.5\n335.3\n477.4\n378.4\n538.4\n88.6\n88.7\n78.7\n78.8\n21.3\n21.2\n\n\nОбеспечение электрической энергией, газом и паром; кондиционирование воздуха\n60654.9\n79318.7\n370.1\n489.3\n415.8\n549.7\n89\n89.0\n79.3\n78.9\n20.7\

Теперь рассмотрим, как будут нарезаны фрагменты, если установить `mode='elements'`. В [документации](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.excel.UnstructuredExcelLoader.html#langchain_community.document_loaders.excel.UnstructuredExcelLoader.load_and_split) сказано, что в этом случае каждый лист становится отдельным элементом. Но после этого происходит еще нарезка внутри одного листа.

In [44]:
loader = UnstructuredExcelLoader('din-zatrat-rs.xlsx', mode='elements')
splitted_xlsx2 = loader.load()

Список получился большим.

In [45]:
len(splitted_xlsx2)

43

Соответственно, каждый фрагмент будет маленьким.

In [46]:
splitted_xlsx2[5]

Document(page_content='Шалаевская Татьяна Юрьевна', metadata={'source': 'din-zatrat-rs.xlsx', 'filename': 'din-zatrat-rs.xlsx', 'last_modified': '2024-06-01T11:26:49', 'page_name': 'Содержание', 'page_number': 1, 'languages': ['rus'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'category': 'Title'})

### Создание векторных представлений и векторного хранилища
Используем в итоге загрузчик с настройками по умолчанию.

Создадим очередное векторное хранилище.

In [47]:
vector_store_xlsx = FAISS.from_documents(splitted_xlsx1, embedding=embedding)

Снова зададим вопросы.

#### Вопрос 4
Какой процент расходов на социальную защиту работников в добыче полезных ископаемых был зафиксирован в 2021 году?

#### Правильный ответ
В 2021 году расходы на социальную защиту работников в добыче полезных ископаемых составляли 22,6%.

In [48]:
vector_store_xlsx.similarity_search(q4)

[Document(page_content='Среднемесячные затраты, рублей\n\nСредние затраты на один оплаченный час, рублей\n\nСредние затраты на один отработанный час, рублей\n\nОтношение отработанного времени к оплаченному,  процентов\n\nДоля прямых затрат на рабочую силу в общих затратах на рабочую силу, процентов\n\nДоля косвенных затрат на рабочую силу в общих затратах на рабочую силу, процентов\n\n\n\n\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n\n\nВсего по обследуемым видам экономической деятельности\n60234.7\n90058.9\n374.3\n563.3\n419.4\n630.4\n89.2\n89.4\n80\n80.2\n20\n19.8\n\n\nДобыча полезных ископаемых\n104076.1\n140967.4\n622.5\n845.1\n722.3\n973.6\n86.2\n86.8\n80.3\n80.0\n19.7\n20.0\n\n\nОбрабатывающие производства\n54154.2\n76419.5\n335.3\n477.4\n378.4\n538.4\n88.6\n88.7\n78.7\n78.8\n21.3\n21.2\n\n\nОбеспечение электрической энергией, газом и паром; кондиционирование воздуха\n60654.9\n79318.7\n370.1\n489.3\n415.8\n549.7\n89\n89.0\n79.3\n78.9\n20.7

#### Вопрос 5
Какой процент от общих затрат на рабочую силу составляла заработная плата в 2005 году по всем обследованным видам экономической деятельности?

#### Правильный ответ
В 2005 году заработная плата составляла 76,6% от общих затрат на рабочую силу по всем обследованным видам экономической деятельности.

In [49]:
vector_store_xlsx.similarity_search(q5)

[Document(page_content='К содержанию\n\nОсновные понятия и определения\n\nЗатраты организаций на рабочую силу - это сумма вознаграждений работников (среднесписочной численности) в денежной и неденежной формах за отработанное и неотработанное время, расходы организации, связанные с обеспечением работников жильем, профессиональным обучением, культурно-бытовым обслуживанием, социальной защитой, включая расходы на пенсионное, медицинское и другие виды страхования, командировочные расходы, а также налоги и сборы, связанные с использованием наемной рабочей силы. В составе затрат на рабочую силу можно выделить две группы затрат: прямые и косвенные.\nПрямые затраты включают расходы, направленные организациями непосредственно на удовлетворение потребностей работников: заработная плата, расходы на жилье, выплаты социального характера, расходы на профессиональное обучение и культурно-бытовое обслуживание, оплата проезда и другие расходы, не отнесенные к приведенным классификационным группам.\nК к

#### Вопрос 6
В каком году был зафиксирован наибольший процент заработной платы в обрабатывающей промышленности и чему он равен?

#### Правильный ответ
Наибольший процент заработной платы в обрабатывающей промышленности был зафиксирован в 2007 году и составил 79,5%.

In [50]:
vector_store_xlsx.similarity_search(q6)

[Document(page_content='Среднемесячные затраты, рублей\n\nСредние затраты на один оплаченный час, рублей\n\nСредние затраты на один отработанный час, рублей\n\nОтношение отработанного времени к оплаченному,  процентов\n\nДоля прямых затрат на рабочую силу в общих затратах на рабочую силу, процентов\n\nДоля косвенных затрат на рабочую силу в общих затратах на рабочую силу, процентов\n\n\n\n\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n2017\n2021.0\n\n\nВсего по обследуемым видам экономической деятельности\n60234.7\n90058.9\n374.3\n563.3\n419.4\n630.4\n89.2\n89.4\n80\n80.2\n20\n19.8\n\n\nДобыча полезных ископаемых\n104076.1\n140967.4\n622.5\n845.1\n722.3\n973.6\n86.2\n86.8\n80.3\n80.0\n19.7\n20.0\n\n\nОбрабатывающие производства\n54154.2\n76419.5\n335.3\n477.4\n378.4\n538.4\n88.6\n88.7\n78.7\n78.8\n21.3\n21.2\n\n\nОбеспечение электрической энергией, газом и паром; кондиционирование воздуха\n60654.9\n79318.7\n370.1\n489.3\n415.8\n549.7\n89\n89.0\n79.3\n78.9\n20.7